In [88]:
import json
from bs4 import BeautifulSoup
import pandas as pd
from eurlex_api import *
from azure_api_test import *

url_api_AML5 ='https://eur-lex.europa.eu/legal-content/FR/TXT/HTML/?uri=CELEX:02015L0849-20240709'
url_api_AML6 = 'https://eur-lex.europa.eu/legal-content/FR/TXT/HTML/?uri=OJ:L_202401640'
url_api_R_AML6 = 'https://eur-lex.europa.eu/legal-content/FR/TXT/HTML/?uri=OJ:L_202401689'
api_result_AML5 = get_api_result(url_api_AML5)
api_result_AML6 = get_api_result(url_api_AML6)
api_result_R_AML6 = get_api_result(url_api_R_AML6)


/opt/homebrew/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eur-lex.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eur-lex.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eur-lex.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [57]:
for elem in api_result_AML5:
        soup = BeautifulSoup(elem['content'], 'html.parser')
        text = get_content_from_html(soup)
#display(HTML(elem['content']))
print(type(elem['content']))


<class 'str'>


In [ ]:
# Chaîne JSON
json_data = {
    "articles": []
}

# Parser le HTML
soup = BeautifulSoup(elem['content'], 'html.parser')

# Trouver toutes les subdivisions
articles = soup.find_all('div', class_='eli-subdivision')

data = []

for article in articles:
    # Vérifier si c'est un article (ID contient 'art_')
    article_id = article.get('id', '')
    if not article_id.startswith('art_'):
        continue
    print(article_id)
    json_data["articles"].append(json.loads(html_json(article.get_text())))
# Nom du fichier où vous voulez sauvegarder le JSON
fichier_json = "articles_AML5.json"

# Écriture dans le fichier
with open(fichier_json, "w", encoding="utf-8") as fichier:
    json.dump(json_data, fichier, ensure_ascii=False, indent=4)
    

art_1
{'message': 'Fonction html_json appelée avec succès', 'html': "\nArticle premier\n\n1.\xa0\xa0\nLa présente directive vise à prévenir l'utilisation du système financier de l'Union aux fins du blanchiment de capitaux et du financement du terrorisme.\n\n\n2.\xa0\xa0\nLes États membres veillent à ce que le blanchiment de capitaux et le financement du terrorisme soient interdits.\n\n\n3.\xa0\xa0\n\nAux fins de la présente directive, sont considérés comme blanchiment de capitaux les\xa0agissements ci-après énumérés, commis intentionnellement:\n\n\na)\xa0\n\n\nla conversion ou le transfert de biens, dont celui qui s'y livre sait qu'ils proviennent d'une activité criminelle ou d'une participation à une activité criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ou d'aider toute personne impliquée dans une telle activité à échapper aux\xa0conséquences juridiques des actes qu'elle a commis;\n\n\n\n\nb)\xa0\n\n\nle fait de dissimuler ou de déguiser la natu

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [55]:
from bs4 import BeautifulSoup
import pandas as pd

json_data = {
    "Table_correspondance": [
    ] 
}

# Parser le HTML
soup = BeautifulSoup(elem['content'], 'html.parser')

# Extraire le titre de l'article
containers = soup.find_all('div', class_='eli-container')

# Extraire les données des paragraphes
for container in containers:
    if "Tableau de correspondance" in container.get_text():
        oj_tables = container.find_all('tr', class_='oj-table')
        for oj_table in oj_tables:
            oj_tbl_arts = oj_table.find_all('p', class_='oj-tbl-txt')
            entry = []
            for oj_tbl_art in oj_tbl_arts:
                text = oj_tbl_art.get_text().strip()  # Extraire et nettoyer le texte
                entry.append(text)
            # Ajouter l'entrée au JSON
            if entry:  # Ajouter uniquement si non vide
                json_data["Table_correspondance"].append(entry)
fichier_json = "tableau_correspondance.json"
with open(fichier_json, "w", encoding="utf-8") as fichier:
    json.dump(json_data, fichier, indent=4, ensure_ascii=False)
print(f"Le fichier JSON a été écrit avec succès : {fichier_json}")


Le fichier JSON a été écrit avec succès : tableau_correspondance.json


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd


# Parser le HTML
soup = BeautifulSoup(elem['content'], 'html.parser')

# Extraire le titre de l'article
article_titles = soup.find_all('p', class_='title-article-norm')

# Extraire les données des paragraphes
data = []
for article_title in article_titles:
    paragraphs = soup.find_all('div', class_='norm', recursive=True)

    for paragraph in paragraphs:
        paragraph_number_tag = paragraph.find('span', class_='no-parag')
        paragraph_number = paragraph_number_tag.text.strip() if paragraph_number_tag else None

        # Vérifier les sous-paragraphes
        sub_paragraphs = paragraph.find_all('div', class_='grid-container')
        if sub_paragraphs:
            for sub in sub_paragraphs:
                sub_number = sub.find('span').text.strip() if sub.find('span') else None
                sub_content = sub.find('div', class_='grid-list-column-2').text.strip() if sub.find('div', class_='grid-list-column-2') else None
                data.append([article_title, paragraph_number, sub_number, sub_content])
        else:
            # Aucun sous-paragraphe trouvé
            content = paragraph.find('div', class_='norm inline-element').text.strip() if paragraph.find('div', class_='norm inline-element') else None
            data.append([article_title, paragraph_number, None, content])

# Convertir en DataFrame pour un affichage structuré
df = pd.DataFrame(data, columns=['Article', 'Paragraphe', 'Sous-paragraphe', 'Contenu'])

# Afficher le tableau
print(df.dropna(subset='Paragraphe'))

In [ ]:
df.to_csv('test.csv')